Job 1, replicate the forward pass of a single MLP layer from GPTNeoXMLP

In [1]:
from modeling_gpt_neox import GPTNeoXForCausalLM
model = GPTNeoXForCausalLM.from_pretrained("trl-internal-testing/tiny-random-GPTNeoXForCausalLM")
from modeling_gpt_neox import GPTNeoXMLP
mlp = GPTNeoXMLP(model.config)

In [2]:
model.state_dict().keys()

odict_keys(['gpt_neox.embed_in.weight', 'gpt_neox.layers.0.input_layernorm.weight', 'gpt_neox.layers.0.input_layernorm.bias', 'gpt_neox.layers.0.post_attention_layernorm.weight', 'gpt_neox.layers.0.post_attention_layernorm.bias', 'gpt_neox.layers.0.attention.bias', 'gpt_neox.layers.0.attention.masked_bias', 'gpt_neox.layers.0.attention.rotary_emb.inv_freq', 'gpt_neox.layers.0.attention.query_key_value.weight', 'gpt_neox.layers.0.attention.query_key_value.bias', 'gpt_neox.layers.0.attention.dense.weight', 'gpt_neox.layers.0.attention.dense.bias', 'gpt_neox.layers.0.mlp.dense_h_to_4h.weight', 'gpt_neox.layers.0.mlp.dense_h_to_4h.bias', 'gpt_neox.layers.0.mlp.dense_4h_to_h.weight', 'gpt_neox.layers.0.mlp.dense_4h_to_h.bias', 'gpt_neox.layers.1.input_layernorm.weight', 'gpt_neox.layers.1.input_layernorm.bias', 'gpt_neox.layers.1.post_attention_layernorm.weight', 'gpt_neox.layers.1.post_attention_layernorm.bias', 'gpt_neox.layers.1.attention.bias', 'gpt_neox.layers.1.attention.masked_bias',

In [3]:
mlp.state_dict()

OrderedDict([('dense_h_to_4h.weight',
              tensor([[-0.1292,  0.0443, -0.0774,  ...,  0.1287,  0.0994,  0.1353],
                      [-0.0972,  0.0236,  0.0603,  ..., -0.0110,  0.1672, -0.0086],
                      [-0.1191,  0.1188, -0.0998,  ...,  0.1715, -0.0931,  0.0332],
                      ...,
                      [-0.1726, -0.1050,  0.0902,  ...,  0.0563,  0.0638, -0.1391],
                      [ 0.1363, -0.1630, -0.1528,  ...,  0.1118, -0.1547, -0.0386],
                      [-0.1348,  0.0011,  0.0667,  ..., -0.1358, -0.0009,  0.0612]])),
             ('dense_h_to_4h.bias',
              tensor([-0.1660, -0.0538, -0.0403,  0.1558, -0.1458, -0.1026,  0.0619, -0.0821,
                      -0.0827, -0.0799, -0.0938, -0.1722,  0.1314,  0.0845, -0.0783,  0.0237,
                       0.1370,  0.0882, -0.0298, -0.0890,  0.1016, -0.1717,  0.1479, -0.0651,
                      -0.0814, -0.1165, -0.0256,  0.0875, -0.0698,  0.0542,  0.0200,  0.1422,
                

In [33]:
# rename the keys to match the mlp
new_state_dict = {}
for k, v in model.state_dict().items():
    if k.startswith("gpt_neox.layers.1.mlp."):
        new_state_dict[k.replace("gpt_neox.layers.1.mlp.", "")] = v

mlp.load_state_dict(new_state_dict)

<All keys matched successfully>

In [34]:
import torch
inputState = torch.randn(2, 32)
mlp(inputState)

tensor([[-4.8557e-03, -7.1405e-03,  7.3479e-04,  1.7308e-03,  1.0328e-02,
          5.5924e-03,  1.7156e-03,  8.4459e-03,  8.2332e-03,  3.5740e-03,
         -8.4398e-04, -3.7490e-03, -2.2467e-03,  5.3495e-03, -3.3868e-03,
         -5.4625e-04,  3.0698e-03, -2.8900e-03, -4.0124e-05, -7.1420e-03,
         -3.3980e-03, -4.9446e-03, -3.6563e-03,  8.5901e-03, -5.9791e-03,
         -2.1923e-03, -1.7515e-03,  3.2583e-03,  9.8841e-03, -3.3359e-03,
          8.8847e-04, -7.3157e-03],
        [ 7.6398e-03,  5.2383e-03,  9.7130e-04,  6.1106e-03, -6.8673e-03,
          4.9993e-03, -1.7223e-03,  6.2101e-03, -7.9022e-03,  1.0678e-02,
          2.7250e-03, -1.6588e-03,  2.5691e-03,  3.7006e-03, -4.1308e-03,
          2.5991e-03, -3.5686e-03,  2.0885e-03, -4.1439e-03,  8.7240e-03,
         -1.2532e-03,  4.4078e-03,  6.3213e-03,  4.7644e-03, -2.6624e-03,
          4.8762e-03, -6.2065e-03,  3.4692e-03, -8.7608e-03,  2.0760e-03,
          7.4033e-04, -7.3633e-03]], grad_fn=<AddmmBackward0>)

In [58]:
import torch.nn as nn
import math

class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.hidden_size, config.intermediate_size, bias=True) # n_embd, 4 * n_embd
        self.c_proj  = nn.Linear(config.intermediate_size, config.hidden_size, bias=True)
        self.dropout = nn.Dropout(0) # no dropout
        self.act = torch.nn.GELU()

    def forward(self, x):
        x = self.c_fc(x)
        x = self.act(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

mlp2 = MLP(model.config)
new_state_dict = {
    "c_fc.weight": model.state_dict()["gpt_neox.layers.1.mlp.dense_h_to_4h.weight"],
    "c_fc.bias": model.state_dict()["gpt_neox.layers.1.mlp.dense_h_to_4h.bias"],
    "c_proj.weight": model.state_dict()["gpt_neox.layers.1.mlp.dense_4h_to_h.weight"],
    "c_proj.bias": model.state_dict()["gpt_neox.layers.1.mlp.dense_4h_to_h.bias"],
}
mlp2.load_state_dict(new_state_dict)


<All keys matched successfully>

In [59]:
mlp2(inputState), mlp(inputState)


(tensor([[-4.8557e-03, -7.1405e-03,  7.3479e-04,  1.7308e-03,  1.0328e-02,
           5.5924e-03,  1.7156e-03,  8.4459e-03,  8.2332e-03,  3.5740e-03,
          -8.4398e-04, -3.7490e-03, -2.2467e-03,  5.3495e-03, -3.3868e-03,
          -5.4625e-04,  3.0698e-03, -2.8900e-03, -4.0124e-05, -7.1420e-03,
          -3.3980e-03, -4.9446e-03, -3.6563e-03,  8.5901e-03, -5.9791e-03,
          -2.1923e-03, -1.7515e-03,  3.2583e-03,  9.8841e-03, -3.3359e-03,
           8.8847e-04, -7.3157e-03],
         [ 7.6398e-03,  5.2383e-03,  9.7130e-04,  6.1106e-03, -6.8673e-03,
           4.9993e-03, -1.7223e-03,  6.2101e-03, -7.9022e-03,  1.0678e-02,
           2.7250e-03, -1.6588e-03,  2.5691e-03,  3.7006e-03, -4.1308e-03,
           2.5991e-03, -3.5686e-03,  2.0885e-03, -4.1439e-03,  8.7240e-03,
          -1.2532e-03,  4.4078e-03,  6.3213e-03,  4.7644e-03, -2.6624e-03,
           4.8762e-03, -6.2065e-03,  3.4692e-03, -8.7608e-03,  2.0760e-03,
           7.4033e-04, -7.3633e-03]], grad_fn=<AddmmBackward0>)

In [62]:

# check that the two mlp's are the same
torch.allclose(mlp2(inputState), mlp(inputState))

True